In [2]:
from pennylane import numpy as np
import pennylane as qml


In [7]:
symbols = ["H", "H"]
coordinates = np.array([0, 0, 0.6614, 0, 0, -0.6614])

H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.2427450126094144) [Z2]
+ (-0.2427450126094144) [Z3]
+ (-0.042072551947439224) [I0]
+ (0.1777135822909176) [Z0]
+ (0.1777135822909176) [Z1]
+ (0.12293330449299361) [Z0 Z2]
+ (0.12293330449299361) [Z1 Z3]
+ (0.16768338855601356) [Z0 Z3]
+ (0.16768338855601356) [Z1 Z2]
+ (0.17059759276836803) [Z0 Z1]
+ (0.1762766139418181) [Z2 Z3]
+ (-0.044750084063019925) [Y0 Y1 X2 X3]
+ (-0.044750084063019925) [X0 X1 Y2 Y3]
+ (0.044750084063019925) [Y0 X1 X2 Y3]
+ (0.044750084063019925) [X0 Y1 Y2 X3]


In [9]:
dev = qml.device("default.qubit", wires=qubits)

electrons = 2
hf = qml.qchem.hf_state(electrons, qubits)

In [14]:
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

In [15]:
@qml.qnode(dev, interface="autograd")
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

In [16]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array(0.0, requires_grad=True)

In [17]:
# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")

Step = 0,  Energy = -1.12799983 Ha
Step = 2,  Energy = -1.13466246 Ha
Step = 4,  Energy = -1.13590595 Ha
Step = 6,  Energy = -1.13613667 Ha
Step = 8,  Energy = -1.13617944 Ha
Step = 10,  Energy = -1.13618736 Ha
Step = 12,  Energy = -1.13618883 Ha

Final value of the ground-state energy = -1.13618883 Ha

Optimal value of the circuit parameter = 0.2089


In [18]:
optimun = angle[-1]

@qml.qnode(dev, interface="autograd")
def cost_fn(param,obs):
    circuit(param, wires=range(qubits))
    return qml.probs(wires=obs)

In [21]:
number_pairs = int( qubits/2 )
value_per_sites = []
for i in range(number_pairs):
    result_down = cost_fn(optimun, obs =[2*i])
    result_up = cost_fn(optimun, obs =[2*i+1])
    value_per_sites.append(result_up[1] + result_down[1])

In [22]:
print( value_per_sites )

[tensor(1.97826969, requires_grad=True), tensor(0.02173031, requires_grad=True)]


In [23]:
number_pairs = int( qubits/2 )
value_per_sites = []
for i in range(number_pairs):
    result_down = cost_fn(optimun, obs =[2*i])
    result_up = cost_fn(optimun, obs =[2*i+1])
    value_per_sites.append(result_up[1] - result_down[1])

In [24]:
print( value_per_sites )

[tensor(0., requires_grad=True), tensor(0., requires_grad=True)]
